In [1]:
# Delete this cell to re-enable tracebacks
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [2]:
# JSON output syntax highlighting
from __future__ import print_function
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML

original_print = print

def json_print(inpt):
    string = str(inpt)
    if string[0] == '{':
        formatter = HtmlFormatter()
        return HTML('<style type="text/css">{}</style>{}'.format(
                    formatter.get_style_defs('.highlight'),
                    highlight(string, JsonLexer(), formatter)))
    else:
        original_print(inpt)

print = json_print

# DataStore API

CTI Python STIX2 features a new interface for pulling and pushing STIX2 content. The new interface consists of DataStore, DataSource and DataSink constructs: a DataSource for pulling STIX2 content, a DataSink for pushing STIX2 content, and a DataStore for pulling/pushing.

The DataStore, DataSource, DataSink (referred to as "DataStore suite") APIs are not referenced directly by a user but are used as base classes, which are then sublcassed into real DataStore suite(s). CTI Python STIX2 provides for the DataStore suites of **FileSystem**, **Memory**, and **TAXII**. Users are also encrouraged subclassing the base Data suite and creating their own custom DataStore suites.

## CompositeDataSource

**CompositeDataSource** is an available controller that can be used as a single interface to a set of defined DataSources. The purpose of this controller is allow for the grouping of **DataSources** and making get/query calls to a set of DataSources in one API call. **CompositeDataSource** can be used to organize/group **DataSources**, federate get()/all_versions()/query() calls, and reduce user code.

**CompositeDataSource** is just a wrapper around a set of defined **DataSources** (e.g. FileSystemSource) that federates get()/all_versions()/query() calls individually to each of the attached **DataSources** , collects the results from each **DataSource** and returns them.

Filters can be attached to **CompositeDataSources** just as they can be done to **DataStores** and **DataSources**. When get()/all_versions()/query() calls are made to the **CompositeDataSource**, it will pass along any query filters from the call and any of its own filters to the attached **DataSources**. To which, those attached **DataSources** may have their own attached filters as well. The effect is that all the filters are eventually combined when the get()/all_versions()/query() call is actually executed within a **DataSource**. 

A **CompositeDataSource** can also be attached to a **CompositeDataSource** for multiple layers of grouped **DataSources**.


### CompositeDataSource API

#### CompositeDataSource Examples


In [1]:
from taxii2client import Collection
from stix2 import CompositeDataSource, FileSystemSource, TAXIICollectionSource

# create FileSystemStore
fs = FileSystemSource("/tmp/stix2_data")

# create TAXIICollectionSource
colxn = Collection('https://test.freetaxii.com:8000/api1/collections/9cfa669c-ee94-4ece-afd2-f8edac37d8fd/')
ts = TAXIICollectionSource(colxn)

# add them both to the CompositeDataSource
cs = CompositeDataSource()
cs.add_data_sources([fs, ts])

# get an object that is only in the filesystem
ta = cs.get('intrusion-set--f3bdec95-3d62-42d9-a840-29630f6cdc1a')
print(ta)

# get an object that is only in the TAXII collection
ind = cs.get('indicator--37a6a5de-a5b9-425a-903a-4ae9cbf1ff3f')
print(ind)


{
    "type": "indicator",
    "id": "indicator--797ae2b5-3f7a-44c5-8ecd-33ba22fdc2b5",
    "created": "2017-10-04T19:27:41.000Z",
    "modified": "2017-10-04T19:27:41.000Z",
    "labels": [
        "malicious-activity"
    ],
    "name": "Emerging Threats - Block Rules - Compromised IPs",
    "pattern": "[ ipv4-addr:value = '98.138.19.88' ]",
    "valid_from": "2017-10-04T19:27:41Z",
    "kill_chain_phases": [
        {
            "kill_chain_name": "lockheed-martin-cyber-kill-chain",
            "phase_name": "delivery"
        }
    ]
}
{
    "type": "indicator",
    "id": "indicator--11913f42-2d52-4b9d-842f-94bf06819a66",
    "created": "2017-10-04T19:27:41.000Z",
    "modified": "2017-10-04T19:27:41.000Z",
    "labels": [
        "malicious-activity"
    ],
    "name": "Emerging Threats - Block Rules - Compromised IPs",
    "pattern": "[ ipv4-addr:value = '98.138.19.88' ]",
    "valid_from": "2017-10-04T19:27:41Z",
    "kill_chain_phases": [
        {
            "kill_chain_name

## Filters

The CTI Python STIX2 **DataStore** suites - **FileSystem**, **Memory** and **TAXII** - all use the **Filters** module to allow for the querying of STIX content. The basic functionality is that filters can be created and supplied everytime to calls to **query()**, and/or attached to a **DataStore** so that every future query placed to that **DataStore** is evaluated against the attached filters, supplemented with any further filters supplied with the query call. Attached filters can also be removed from **DataStores**.

Filters are very simple, as they consist of a field name, comparison operator and an object property value (i.e. value to compare to). Currently, CTI Python STIX2 supports **ONLY** STIX 2 object common properties and TAXII2 Filtering parameters for fields to filter on:

Fields

(STIX2 Object Common Properties)

* created
* created_by_ref
* external_references.source_name
* external_references.description
* external_references.url
* external_references.external_id
* granular_markings.marking_ref
* granular_markings.selectors
* id
* labels
* modified
* object_marking_refs
* revoked
* type

(TAXII2 filter fields)

* added_after
* match[id]
* match[type]
* match[version]

Supported operators on above properties:

* =
* !=
* in
* >
* < 
* ```>=```
* <=

Value types of the common property values must be one of these (python) types:

* bool
* dict
* float
* int
* list
* str
* tuple

### Filter Examples

In [ ]:
import sys
from stix2 import Filter

# create filter for STIX objects that have external references to MITRE ATT&CK framework
f = Filter("external_references.source_name", "=", "mitre-attack")

# create filter for STIX objects that are not of SDO type Attack-Pattnern
f1 = Filter("type", "!=", "attack-pattern")

# create filter for STIX objects that have the "threat-report" label
f2 = Filter("labels", "in", "threat-report")

# create filter for STIX objects that have been modified past the timestamp
f3 = Filter("modified", ">=", "2017-01-28T21:33:10.772474Z")

# create filter for STIX objects that have been revoked
f4 = Filter("revoked", "=", True)

For Filters to be applied to a query, they must be either supplied with the query call or attached a **DataStore**, more specifically to **DataSource** whether that **DataSource** is a part of a **DataStore** or stands by itself. 

In [ ]:
from stix2 import MemoryStore, FileSystemStore, FileSystemSource

fs = FileSystemStore("/home/michael/Desktop/sample_stix2_data")
fs_source = FileSystemSource("/home/michael/Desktop/sample_stix2_data")

# attach filter to FileSystemStore
fs.source.filters.add(f)

# attach multiple filters to FileSystemStore
fs.source.filters.update([f1,f2])

# can also attach filters to a Source
# attach multiple filters to FileSystemSource
fs_source.filters.update([f3, f4])


mem = MemoryStore()

# As it is impractical to only use MemorySink or MemorySource,
# attach a filter to a MemoryStore
mem.source.filters.add(f)

# attach multiple filters to a MemoryStore
mem.source.filters.update([f1,f2])